## Google Photos API

## Create virtualenv and install required packages

1. Open terminal and navigate to your working directory

2. Create a virtual environment `python3 -m venv venv`

3. Add your virtual environment to Jupyter: `python3 -m ipykernel install --user --name=venv`

4. Start jupyter notebook or jupyter lab: `jupyter lab .`

## Enable Google API 



5. Enable Google Photos API Service

   1. Go to the [Google API Console](https://console.cloud.google.com/). 
   2. From the menu bar, select a project or create a new project.
      ![](img/create_a_project_google.png)
   3. To open the Google API Library, from the Navigation menu, select APIs & Services > Library. 
   4. Search for "Google Photos Library API". Select the correct result and click "enable". If its already enabled, click "manage"
   5. Afterwards it will forward you to the "Photos API/Service details" page (https://console.cloud.google.com/apis/credentials)


6. Create API/OAuth credentials

   1. On the left side at the Google Photos API Service page click on Credentials
   2. Click on "Create Credentials" and create a OAuth client ID
   3. As application type I am choosing "Desktop app" and give your client you want to use to call the API a name
   4. Download the JSON file to the created credentials, rename it to "client_secret.json" and save it in the folder "credentials"


7. Configure "OAuth consent screen" ([Source](https://stackoverflow.com/questions/65184355/error-403-access-denied-from-google-authentication-web-api-despite-google-acc))

   1. Go back to the Photos API Service details page and click on "[OAuth consent screen](https://console.cloud.google.com/apis/credentials/consent)" on the left side (below "Credentials") 
   2. Add a Test user: Use the email of the account you want to use for testing the API call
   

## Install required packages

In [1]:
%%capture capt 
#saves the output to variable capt, to print output capt.stdout, capt.stderr
!pip install google_auth_oauthlib
!pip install pip install google-api-python-client

In [2]:
import pandas as pd

# import customized modules
import sys
import os

import sys
sys.path.append(r'./helper_scripts')
from google_photos_api import GooglePhotosApi

## Utilize Google Photos Library API

5. Use the customized funtions from "google_photos_api.py" to create a service for the first time:

    1. Initialize GooglePhotosApi `google_photos_api = GooglePhotosApi()`

    2. Create Service using the `client_secret.json` file: `service = google_photos_api.create_service()`
        
        
       <b>Calling the API for the first time:</b>
       1. Google will ask you if you want to grant the App the required permissions you defined with the scope:
       ![](img/sign_in_google_acc.png)
       2. Since its just a test app at the moment, Google will make you aware of that > Click on "Continue"
       3. Once you granted the app the required permissions, you will see a "token_......pickle" file created in the folder "credentials". This token file will be used for future calls.

In [33]:
import pickle
import os
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
#from googleapiclient.http import MediaFileUpload
from google.auth.transport.requests import Request
import requests

class GooglePhotosApi:
    def __init__(self,
                 api_name = 'photoslibrary',
                 client_secret_file= r'./credentials/client_secret.json',
                 api_version = 'v1',
                 scopes = ['https://www.googleapis.com/auth/photoslibrary']):
        '''
        Args:
            client_secret_file: string, location where the requested credentials are saved
            api_version: string, the version of the service
            api_name: string, name of the api e.g."docs","photoslibrary",...
            api_version: version of the api

        Return:
            service:
        '''

        self.api_name = api_name
        self.client_secret_file = client_secret_file
        self.api_version = api_version
        self.scopes = scopes
        self.cred_pickle_file = f'./credentials/token_{self.api_name}_{self.api_version}.pickle'

        self.cred = None

    def run_local_server(self):
        # is checking if there is already a pickle file with relevant credentials
        if os.path.exists(self.cred_pickle_file):
            with open(self.cred_pickle_file, 'rb') as token:
                self.cred = pickle.load(token)

        # if there is no pickle file with stored credentials, create one using google_auth_oauthlib.flow
        if not self.cred or not self.cred.valid:
            if self.cred and self.cred.expired and self.cred.refresh_token:
                self.cred.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.client_secret_file, self.scopes)
                self.cred = flow.run_local_server()

            with open(self.cred_pickle_file, 'wb') as token:
                pickle.dump(self.cred, token)
        
        return self.cred


In [120]:
# initialize photos api and create service
google_photos_api = GooglePhotosApi()
creds = google_photos_api.run_local_server()

### Get list of all items on a specfic day

Use requests python module to send http requests to the Media Items API

Sources:
* https://stackoverflow.com/questions/56294506/mediaitems-search-next-returns-400
* https://developers.google.com/photos/library/reference/rest/v1/mediaItems/search?apix_params=%7B%22resource%22%3A%7B%22filters%22%3A%7B%22dateFilter%22%3A%7B%22dates%22%3A%5B%7B%22day%22%3A1%2C%22month%22%3A1%2C%22year%22%3A2022%7D%5D%7D%7D%7D%7D

In [122]:
import json
import requests

def get_response_from_medium_api(year, month, day):
    url = 'https://photoslibrary.googleapis.com/v1/mediaItems:search'
    payload = {
                  "filters": {
                    "dateFilter": {
                      "dates": [
                        {
                          "day": day,
                          "month": month,
                          "year": year
                        }
                      ]
                    }
                  }
                }
    headers = {
        'content-type': 'application/json',
        'Authorization': 'Bearer {}'.format(creds.token)
    }
    
    res = requests.request("POST", url, data=json.dumps(payload), headers=headers)
    
    return(res)

In [133]:
date.today()

datetime.date(2022, 4, 17)

In [138]:
import pandas
from datetime import date, timedelta

sdate = date(2018,3,22)   # start date
edate = date.today()

date_list = pandas.date_range(sdate,edate-timedelta(days=1),freq='d')
date_list

DatetimeIndex(['2018-03-22', '2018-03-23', '2018-03-24', '2018-03-25',
               '2018-03-26', '2018-03-27', '2018-03-28', '2018-03-29',
               '2018-03-30', '2018-03-31',
               ...
               '2022-04-07', '2022-04-08', '2022-04-09', '2022-04-10',
               '2022-04-11', '2022-04-12', '2022-04-13', '2022-04-14',
               '2022-04-15', '2022-04-16'],
              dtype='datetime64[ns]', length=1487, freq='D')

In [128]:
response = get_response_from_medium_api(year=2021, month=1, day=1)

#response = requests.request("GET", url, headers=headers)
#response.to_json()
media_items_df = pd.DataFrame()

for item in response.json()['mediaItems']:
    item_df = pd.concat([test, pd.DataFrame(item)])
    item_df = item_df[item_df.index == 'creationTime']
    #print(item)
    #test.append(pd.DataFrame(item))
    media_items_df = pd.concat([media_items_df, item_df])
media_items_df
media_items_df.rename(columns={"mediaMetadata": "creationTime"})
media_items_df
#media_items_df.set_index('creationTime')
#item = pd.DataFrame(response.json()['mediaItems'][1])
#item = item[item.index == 'creationTime']

,id,productUrl,baseUrl,mimeType,mediaMetadata,filename
creationTime,AIi7vOhTgt_ADdnSpXMgUEcv6IDdHmCIphIsoFYpqXJAbM...,https://photos.google.com/lr/photo/AIi7vOhTgt_...,https://lh3.googleusercontent.com/lr/AFBm1_ZZ4...,image/jpeg,2021-01-01T18:50:38Z,IMG-20210101-WA0003.jpg
creationTime,AIi7vOhTgt_ADdnSpXMgUEcv6IDdHmCIphIsoFYpqXJAbM...,https://photos.google.com/lr/photo/AIi7vOhTgt_...,https://lh3.googleusercontent.com/lr/AFBm1_ZZ4...,image/jpeg,2021-01-01T18:50:38Z,IMG-20210101-WA0003.jpg
creationTime,AIi7vOiPSVG_e1dGz4wZM8UkIqwKp803RqwTakvu7Ze_lq...,https://photos.google.com/lr/photo/AIi7vOiPSVG...,https://lh3.googleusercontent.com/lr/AFBm1_YPG...,image/jpeg,2021-01-01T18:50:38Z,IMG-20210101-WA0004.jpg
creationTime,AIi7vOhTgt_ADdnSpXMgUEcv6IDdHmCIphIsoFYpqXJAbM...,https://photos.google.com/lr/photo/AIi7vOhTgt_...,https://lh3.googleusercontent.com/lr/AFBm1_ZZ4...,image/jpeg,2021-01-01T18:50:38Z,IMG-20210101-WA0003.jpg
creationTime,AIi7vOhTgt_ADdnSpXMgUEcv6IDdHmCIphIsoFYpqXJAbM...,https://photos.google.com/lr/photo/AIi7vOhTgt_...,https://lh3.googleusercontent.com/lr/AFBm1_ZZ4...,image/jpeg,2021-01-01T18:50:38Z,IMG-20210101-WA0003.jpg
...,...,...,...,...,...,...
creationTime,AIi7vOilfMPWRa5ukZuPQDG4-3yQiGQW7lq1Xktt2bIcuH...,https://photos.google.com/lr/photo/AIi7vOilfMP...,https://lh3.googleusercontent.com/lr/AFBm1_ZNI...,image/jpeg,2021-01-01T18:50:37Z,IMG-20210101-WA0002.jpg
creationTime,AIi7vOha3gZjK8FcBUgHd9ZUQh_nbHZBaONZfxrSjjkBHr...,https://photos.google.com/lr/photo/AIi7vOha3gZ...,https://lh3.googleusercontent.com/lr/AFBm1_avb...,image/jpeg,2021-01-01T13:01:43Z,Screenshot_20210101_140143_com.android.chrome.jpg
creationTime,AIi7vOjpOsxi16X-3ekBmmfTvnOHozmoaaeC7rSgVEOh8z...,https://photos.google.com/lr/photo/AIi7vOjpOsx...,https://lh3.googleusercontent.com/lr/AFBm1_bU2...,image/jpeg,2021-01-01T01:29:20Z,IMG-20210101-WA0001.jpg
creationTime,AIi7vOifoJsLf7WLdk_ttLmeDcX85PJ4m3IBE1pu_507d_...,https://photos.google.com/lr/photo/AIi7vOifoJs...,https://lh3.googleusercontent.com/lr/AFBm1_bNB...,image/jpeg,2020-12-31T23:27:18Z,IMG-20210101-WA0000.jpg


In [91]:
test

""


In [125]:
response.json()['mediaItems'][0]

{'id': 'AIi7vOhTgt_ADdnSpXMgUEcv6IDdHmCIphIsoFYpqXJAbM_G7RdS1TrQ_23j6FmarihWbg-0q4DAbiFp_znm5MX_7lEFy-y5Cw',
 'productUrl': 'https://photos.google.com/lr/photo/AIi7vOhTgt_ADdnSpXMgUEcv6IDdHmCIphIsoFYpqXJAbM_G7RdS1TrQ_23j6FmarihWbg-0q4DAbiFp_znm5MX_7lEFy-y5Cw',
 'baseUrl': 'https://lh3.googleusercontent.com/lr/AFBm1_ZZ4khdEAT0THNkaJJTtVVUSFAo_OTRIV-JebbdCy1jW4xelcLHdoqE9_7VWCE6GmId-rZ4HM5aB6Uu3uiSfcV4sPBmf-a-CMrpskzeVU76Lj0Xn_di6d_Q2zPZriZO7B-Uu2yqf1njURBKUEmEy_U_0jNYTTJczJMBIKD3N5N_S_iUnb4X4y0HATtnbcQX54kUy2YhFCo79y57wjYJSkFteoda3eiLaETxmorcqzDZTBgg1qdin2Hg_i0gW8gs_IFBq5wH-x6RNV1c3WH9Hngoo8Awq2twnJqByQw0psoX1zhyjdaNiruX1pkFDKKRzT2Pwwi-70yG_JJ61urKdLvjxyeGPUPZXo6ZIcXie5NvMSYJ42FPmrkldmfkr2m8VbHBSNNB22TuF63KkYR4hNBx6tbYamxtZdiCB-gEbaREhjkOABwjt9oT5Ze1kowmtogXNaO1KEa2uX_h4Rjz2Sinbn-aZ7O47ai10EfvPJKVgXbadIcYTrqX_6aSd7lO-sSqx1tXzyXN1tpPvqTQqFR76aOyOkuEzd5qlQ_dVbXE4v35Q-C-aXr-weJVpGCtyHRbXi5EcZvxtG40xe6pyQzTkBSZIvuVRVGJnbPedfx34PH-vLsOdIlcCFzt-Hl2sGigU_QjuAvv_P3FmVtLbPOQTLJhBI3oAfzg5NIOfbTmh

In [28]:
import re

def extract_
    search_id = r'mimeType": "'
    results = re.findall(search_id + r'[^"]*', res.text)
    results_df = pd.DataFrame(results)
    results_df.columns = ['mimeType']

    # delete field description
    results_df['mimeType'] = results_df['mimeType'].apply(lambda row: row.replace(search_id, ""))
    results_df['mimeType']

0     image/jpeg
1     image/jpeg
2     image/jpeg
3     image/jpeg
4     image/jpeg
5     image/jpeg
6     image/jpeg
7     image/jpeg
8     image/jpeg
9     image/jpeg
10    image/jpeg
Name: mimeType, dtype: object

In [26]:
results_df['mimeType'] = results_df['mimeType'].apply(lambda row: row.replace(search_id, ""))
results_df['mimeType']

0     image/jpeg
1     image/jpeg
2     image/jpeg
3     image/jpeg
4     image/jpeg
5     image/jpeg
6     image/jpeg
7     image/jpeg
8     image/jpeg
9     image/jpeg
10    image/jpeg
Name: mimeType, dtype: object

In [40]:
response = get_response_from_medium_api(year=2021, month=2, day=15)

## Download images

In [ ]:
import requests
url = media_items['mediaItems'][0]['productUrl']
respons = requests.get(url)
file_name = media_items['mediaItems'][0]['filename']
with open(os.path.join(destination_folder, file_name), 'wb') as f:
    f.write(response.content)
    f.close()

## Upload files to AWS

In [ ]:
import boto3

client = boto3.client('s3', region_name='us-west-2')

client.upload_file('images/image_0.jpg', 'mybucket', 'image_0.jpg')